# Model-Based Fault Diagnosis and Security of CPS

In [ ]:
display("text/javascript", """MathJax.Hub.Config({TeX: { equationNumbers: { autoNumber: "AMS" } }});""")

In [ ]:
using ControlSystems

In [ ]:
srand(1234);

## Sensor Faults

In [ ]:
# Example in S. Sundaram lecture notes, ch. 4
A = 0.39 * [1 1 0 1; 0 1 1 0; 1 0 1 1; 0 0 1 1]
C = [1 0 0 0; 0 1 0 0; 0 1 1 0]
sys = ss(A,zeros(4,1),C,zeros(3,1),-1)

In [ ]:
abs.(eigvals(A))

In [ ]:
# Observability matrix
C1 = Array(C[1,:]')
rank(obsv(A,C1))

In [ ]:
L = place(A',C1',[0,0,0,0])'  # place the 4 eigenvalues at 0

In [ ]:
A - L*C1

In [ ]:
nsteps = 15
x_true = zeros(size(A,1),nsteps)
y_meas = zeros(size(C,1),nsteps-1)
x_true[:,1] = randn(4)
x_est = zeros(x_true)
y_est = zeros(y_meas-1)
residues = zeros(size(C,1),nsteps-1)
fault = [1;0;0]   # additive constant fault vector - choose ona at a time
for k=1:nsteps-1
    y_meas[:,k] = C * x_true[:,k] + fault
    x_true[:,k+1] = A * x_true[:,k] 
    x_est[:,k+1] = A * x_est[:,k] + L*(y_meas[1,k] - C1*x_est[:,k]) 
    y_est[:,k] = C * x_est[:,k] 
    residues[:,k] = y_meas[:,k] - y_est[:,k]
end

In [ ]:
using Plots
plotlyjs()
plot(residues')

## Parity Space Method

In [ ]:
A = [1 1 0 1; 0 1 1 0; 1 0 1 1; 0 0 1 1]
C = [1 0 0 0; 0 1 0 0; 0 1 1 0]
Bf = [1 0; 0 1; 0 1; 1 1]
Bd = [0; 1; 0; 1];

In [ ]:
O3 = obsv(A,C);

In [ ]:
function outputMatrices(A,B,C,D,order=size(A,1)-1)
    O = C  # Observability matrix
    M = D  # Invertibility matrix
    for i=1:order
        M = [D zeros(size(D,1),size(M,2)); O*B M]
        O = [C; O*A]
    end
    return(O,M)
end

In [ ]:
(O,Md) = outputMatrices(A,Bd,C,zeros(size(C,1),size(Bd,2)));

In [ ]:
V = nullspace([O Md]')'  # parity space

In [ ]:
(O,Mf) = outputMatrices(A,Bf,C,zeros(size(C,1),size(Bf,2)));

In [ ]:
P = V*Mf

Because the last two columns are 0, we cannot detect any components of a fault occuring at k=3.